<a href="https://colab.research.google.com/github/jinwu99/Text-mining-and-Sentiment-analysis/blob/main/Crawling/Crawling_from_CGV_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab에서는 chromedriver설치 관련 오류해결이 쉽지 않아서
# local jupyter notebook으로 시행하였음.
!pip install Selenium
!pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "http://www.cgv.co.kr/movies/detail-view/?midx=79902"
driver.get(url)

In [ ]:
# 당시 총 리뷰 수는 36233개, 한 페이지에 6개의 리뷰
start = 1
page_num = 36233//6 - start
score_list = []

for page_no in range(start,page_num+start):
    try:
        page_ul = driver.find_element('id','paging_point') # 페이지 포인트 코드 추출
        page_a = page_ul.find_element(By.LINK_TEXT,str(page_no))
        page_a.click()
        time.sleep(1) # 페이지 로딩까지의 시간 두기

        # 작성자 이름
        writers = driver.find_elements(By.CLASS_NAME,'writer-name')
        writer_list += [writer.text for writer in writers]
        # 리뷰
        reviews = driver.find_elements(By.CLASS_NAME,'box-comment')
        review_list += [ review.text for review in reviews ]
        # 작성날짜
        dates = driver.find_elements(By.CLASS_NAME,'day')
        date_list += [date.text for date in dates]

        # 골든에그지수
        good = driver.find_elements(By.CSS_SELECTOR,'span.egg-icon.good')
        all = driver.find_elements(By.CSS_SELECTOR,'span.egg-icon')
        tmp_score = []
        for score in all:
            if score in good:
                tmp_score.append(1)
            else:
                tmp_score.append(0)
        score_list += tmp_score

        cgv_review = pd.DataFrame({"id" : writer_list,
                                   "document" : review_list,
                                   "Date" : date_list,
                                   "label" : score_list})
        cgv_review.to_csv('cgv_review.csv',
                   index=False,
                   encoding='utf-8-sig')

        # 잘 돌아가는지 중간체크용
        print('stored page : ',len(score_list)/6)

        if page_no % 10 == 0: # 10이상의 값을 만났을 때
            if page_no==10 :
                    page_ul = driver.find_element('id','paging_point')
                    next_button = page_ul.find_element(By.CLASS_NAME,"paging-side")
                    next_button.click()
                    time.sleep(1)
            else :
                    page_ul = driver.find_element('id','paging_point')
                    next_button = page_ul.find_elements(By.CLASS_NAME,"paging-side")
                    next_button[2].click()
                    time.sleep(1)

    except NoSuchElementException:
            print("불러올 페이지가 없습니다.")
            break